# Example Usage
This notebook covers example use cases of how the `satellite_czml` package can be used.  The images in this notebook are not the direct output of each cell.  They are taken from the Plotly Dash app that runs at the very end of this notebook.

More information can be found here: [https://github.com/cassova/satellite-czml](https://github.com/cassova/satellite-czml)

In [1]:
import urllib

url = 'https://celestrak.com/NORAD/elements/stations.txt'
tles = [l.decode("utf-8").strip() for l in urllib.request.urlopen(url).readlines()]
tle_list = [[tles[i],tles[i+1],tles[i+2]] for i,_ in enumerate(tles) if i%3==0]
small_tle_list = tle_list[:4]
small_tle_list

[['ISS (ZARYA)',
  '1 25544U 98067A   21016.23305200  .00001366  00000-0  32598-4 0  9992',
  '2 25544  51.6457  14.3113 0000235 231.0982 239.8264 15.49297436265049'],
 ['KESTREL EYE IIM (KE2M)',
  '1 42982U 98067NE  21016.10657201  .00026135  00000-0  14287-3 0  9999',
  '2 42982  51.6330 269.7313 0003870 174.0454 186.0594 15.79671909184560'],
 ['DELLINGR (RBLE)',
  '1 43021U 98067NJ  21016.08538968  .00031421  00000-0  15505-3 0  9997',
  '2 43021  51.6303 266.6367 0003507 141.7069 218.4183 15.81804201180375'],
 ['TEMPEST-D',
  '1 43547U 98067NV  21016.13692563  .00010875  00000-0  87195-4 0  9993',
  '2 43547  51.6351 313.7126 0006282 124.9571 235.2018 15.71624838142830']]

## Simple Example
`tle_list` must be a list of lists even if it only contains one TLE entry.

In [2]:
single_tle = [tle_list[0]]
single_tle

[['ISS (ZARYA)',
  '1 25544U 98067A   21016.23305200  .00001366  00000-0  32598-4 0  9992',
  '2 25544  51.6457  14.3113 0000235 231.0982 239.8264 15.49297436265049']]

In [3]:
from satellite_czml import satellite_czml

single_czml = satellite_czml(tle_list=single_tle).get_czml()

![Simple Example](screenshots/simple_czml.png "Simple Satellite Example")

## Multiple Satellite Example

In [4]:
multiple_czml = satellite_czml(tle_list=small_tle_list).get_czml()

![Multiple Example](screenshots/multiple_czml.png "Multiple Satellite Example")

## Specifying Argument Lists
`tle_list` can be either contain lists of 3 elements (which include name as the first element) or as 2 elements (only the TLE data).  If using the latter, `name_list` must also be provided.

Each list must be the same length.

In [5]:
from datetime import datetime, timedelta
import random

name_list = [t[0] for t in small_tle_list] 
description_list = ['Station: ' + t[0] for t in small_tle_list]
color_list = [[random.randrange(256) for x in range(3)] for x in range(len(small_tle_list))]
size_list = [7] * len(small_tle_list)

czml_obj = satellite_czml(tle_list=small_tle_list, name_list=name_list, description_list=description_list,
                          color_list=color_list, speed_multiplier=1, use_default_image=False,
                          marker_scale_list=size_list, show_label=False, show_path=False,
                          ignore_bad_tles=True)
multiple_czml_p = czml_obj.get_czml()

![Specifying Arguments Example](screenshots/multiple_czml_p.png "Specifying Arguments Example")

## Create One-by-One
Instead of generating several lists to pass to `satellite_czml`, you can create a single list of `satellite` objects.

In [6]:
from satellite_czml import satellite

multiple_sats=[]
for tle in small_tle_list:
    sat = satellite(tle,
                    description='Station: ' + tle[0],
                    color = [random.randrange(256) for x in range(3)],
                    marker_scale=12,
                    use_default_image=False,
                    start_time=datetime.strptime('2020-01-01 00:00:00','%Y-%m-%d %H:%M:%S'),
                    end_time=datetime.strptime('2020-01-01 01:00:00','%Y-%m-%d %H:%M:%S'),
                    show_label=True,
                    show_path=True,
                   )
    multiple_sats.append(sat)
    

czml_obj = satellite_czml(satellite_list=multiple_sats)
multiple_czml_c = czml_obj.get_czml()

![Create One-by-One Example](screenshots/multiple_czml_c.png "Create One-by-One Example")

## Modifying a Satellite

In [7]:
last_sat_key = list(czml_obj.satellites.keys())[-1]
last_sat = czml_obj.satellites[last_sat_key]

# Change the path to display the orbit path
last_sat.build_path(rebuild=True,
                    show=True,
                    color=[255, 255, 0, 127],
                    width=3
                   )

# Change the label look
last_sat.build_label(rebuild=True,
                     show=True,
                     font='12pt Arial',
                     color=[255, 255, 0, 200],
                     outlineColor=[0, 0, 0, 127],
                     outlineWidth=3,
                    )

# Change the marker (billboard)
last_sat.build_marker(rebuild=True,
                      size=18,
                      outlineColor=[0, 0, 0, 128],
                     )

modified_czml = czml_obj.get_czml()

![Modifying Example](screenshots/modified_czml.png "Modifying Example")

## Display in Cesium using Plotly Dash
Replace `YOUR_CESIUM_API_KEY` with your API key from [Cesium](https://cesium.com/) or add to a file `cesium.key` in the same folder as the notebook.

In [ ]:
#apikey = 'YOUR_CESIUM_API_KEY'
apikey = open('cesium.key').read().strip()

In [ ]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output


external_css = ['https://cesium.com/downloads/cesiumjs/releases/1.76/Build/Cesium/Widgets/widgets.css']

external_scripts = [{'src':'https://cesium.com/downloads/cesiumjs/releases/1.76/Build/Cesium/Cesium.js'}]

app = dash.Dash(__name__,
                title='satellite_czml',
                external_scripts=external_scripts,
                external_stylesheets=external_css)

app.layout = html.Div(children=[
    dcc.Dropdown(id='choose_czml_dd',
                 options=[
                     {'label': 'Single CZML', 'value': 'single_czml'},
                     {'label': 'Multiple CZML', 'value': 'multiple_czml'},
                     {'label': 'Specified Arguments CZML', 'value': 'multiple_czml_p'},
                     {'label': 'Create One-by-One CZML', 'value': 'multiple_czml_c'},
                     {'label': 'Modified CZML', 'value': 'modified_czml'}
                 ],
                 value='single_czml'
                ),
    html.Div(id='cesiumContainer'),
    html.Div(id='czml', style={'display': 'none'}),
    html.Div(id='apikey', style={'display': 'none'}, children=apikey)
])

@app.callback(
    Output('czml', 'children'),
    Input('choose_czml_dd', 'value'))
def update_satellite_filter(v):
    d = {'single_czml':single_czml,
         'multiple_czml':multiple_czml,
         'multiple_czml_p':multiple_czml_p,
         'multiple_czml_c':multiple_czml_c,
         'modified_czml':modified_czml
        }
    return d[v]

app.clientside_callback('''
function(id, czml, apikey) {
    // Create the Cesium Viewer
    if (!window.viewer) {
        Cesium.Ion.defaultAccessToken = apikey;
        window.viewer = new Cesium.Viewer(id,{
            shouldAnimate: true,
        });
        window.viewer.scene.globe.enableLighting = true;
    }

    // Update the Cesium Viewer
    if (czml) {
        window.viewer.dataSources.removeAll();
        czmlJson = JSON.parse(czml);
        window.viewer.dataSources.add(
            Cesium.CzmlDataSource.load(czmlJson)
        );
    }

    return true;
}''',
    Output('cesiumContainer', 'data-done'),
    Input('cesiumContainer', 'id'),
    Input('czml', 'children'),
    Input('apikey', 'children')
)

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)